# Movies Analysis

## Imports

In [1]:
# Base Imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Data

In [2]:
basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"
akas_url="https://datasets.imdbws.com/title.akas.tsv.gz"
ratings_url="https://datasets.imdbws.com/title.ratings.tsv.gz"

In [3]:
basics = pd.read_csv(basics_url,sep='\t', low_memory=False)

In [4]:
akas = pd.read_csv(akas_url,sep='\t', low_memory=False)

In [5]:
ratings = pd.read_csv(ratings_url,sep='\t', low_memory=False)

In [6]:

#basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)

In [7]:
#akas = pd.read_csv("Data/title_akas.csv.gz", low_memory = False)

In [8]:
#ratings = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)

## Basics Preprocessing

In [9]:
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
...,...,...,...,...,...,...,...,...,...
9156710,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2010,\N,\N,"Action,Drama,Family"
9156711,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010,\N,\N,"Action,Drama,Family"
9156712,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,\N,\N,"Action,Drama,Family"
9156713,tt9916856,short,The Wind,The Wind,0,2015,\N,27,Short


Notes:
* Replace "\N" with np.nan
* Eliminate movies that are null for runtimeMinutes
* Eliminate movies that are null for genre
* keep only titleType == movie
* keep startYear 2000-2022
* Eliminate movies that include "Documentary" in genre (see tip below)

In [10]:
#scanning dtypes
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9156715 entries, 0 to 9156714
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 628.7+ MB


> Noticed an issue with 'runtimeMinutes'. Some of the records have text values, pretty sure this is supposed to be a datetime dtype.

In [11]:
#replacing \N to np.nan
basics.replace({'\\N':np.nan}, inplace=True)

In [12]:
#dropping NaNs
basics = basics.dropna(axis=0, how='any', thresh=None, subset=['runtimeMinutes', 'genres'], inplace=False)

In [13]:
#instantiating filters for preproc
basics_filtered = basics[(basics.titleType == "movie") & ((basics['startYear'] >= "2000") & (basics['startYear'] <= "2022"))]

In [14]:
#verifying filters
basics_filtered.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
13079,tt0013274,movie,Istoriya grazhdanskoy voyny,Istoriya grazhdanskoy voyny,0,2021,NaN,133,Documentary
34790,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61089,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
66304,tt0067683,movie,Workers '71: Nothing About Us Without Us,Robotnicy 1971 - Nic o nas bez nas,0,2006,NaN,47,Documentary
67635,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama


In [15]:
#excluding movies with 'documentary'
is_documentary = basics_filtered['genres'].str.contains('documentary',case=False)
basics_cleaned = basics_filtered[~is_documentary]

In [16]:
#verifying 'documentary' dropped
basics_cleaned.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34790,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61089,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
67635,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
77929,tt0079644,movie,November 1828,November 1828,0,2001,NaN,140,"Drama,War"
86766,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"


## AKAs Preprocessing

In [17]:
#replacing \N to np.nan
akas.replace({'\\N':np.nan}, inplace=True)

In [18]:
#checking
akas

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,NaN,imdbDisplay,NaN,0
1,tt0000001,2,Carmencita,DE,NaN,NaN,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,NaN,imdbDisplay,NaN,0
3,tt0000001,4,Καρμενσίτα,GR,NaN,imdbDisplay,NaN,0
4,tt0000001,5,Карменсита,RU,NaN,imdbDisplay,NaN,0
...,...,...,...,...,...,...,...,...
32874312,tt9916852,5,Episódio #3.20,PT,pt,NaN,NaN,0
32874313,tt9916852,6,Episodio #3.20,IT,it,NaN,NaN,0
32874314,tt9916852,7,एपिसोड #3.20,IN,hi,NaN,NaN,0
32874315,tt9916856,1,The Wind,DE,NaN,imdbDisplay,NaN,0


In [19]:
#US entries only
akas_us = akas['region'] == 'US'

In [20]:
#variable for US only
akas_cleaned = akas.loc[akas_us]

In [21]:
#checking
akas_cleaned

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0
...,...,...,...,...,...,...,...,...
32873989,tt9916702,1,Loving London: The Playground,US,NaN,NaN,NaN,0
32874026,tt9916720,10,The Demonic Nun,US,NaN,tv,NaN,0
32874028,tt9916720,12,The Nun 2,US,NaN,imdbDisplay,NaN,0
32874045,tt9916756,1,Pretty Pretty Black Girl,US,NaN,imdbDisplay,NaN,0


## Ratings Preprocessing

In [22]:
#replacing \N to np.nan
ratings.replace({'\\N':np.nan}, inplace=True)

In [23]:
#checking
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1905
1,tt0000002,5.9,256
2,tt0000003,6.5,1702
3,tt0000004,5.7,168
4,tt0000005,6.2,2517


## Filtering Basics on AKAs

In [24]:
#filter basics table down to US only
keepers =basics_cleaned['tconst'].isin(akas_cleaned['titleId'])
keepers

34790       True
61089       True
67635       True
77929      False
86766       True
           ...  
9156387     True
9156396     True
9156435    False
9156480     True
9156564    False
Name: tconst, Length: 142175, dtype: bool

In [25]:
#filtering on basics cleaned
basics_cleaned = basics_cleaned[keepers]
basics_cleaned

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34790,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61089,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
67635,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
86766,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
92731,tt0094859,movie,Chief Zabu,Chief Zabu,0,2016,NaN,74,Comedy
...,...,...,...,...,...,...,...,...,...
9155851,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019,NaN,74,Drama
9156247,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019,NaN,97,"Comedy,Drama,Fantasy"
9156387,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,NaN,51,Drama
9156396,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller"


In [26]:
#importing filtered dataframes to Data folder
#import os
#os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
#os.listdir("Data/")

In [27]:
#basics compressed to csv
#basics_cleaned.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

In [28]:
#akas compressed to csv
#akas_cleaned.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)

In [29]:
#ratings compressed to csv
#ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)